In [9]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, pauli_error

import numpy as np

### setup stuff

In [10]:
NumberOfShots = 10

# this generates a random, connected graph
NumberOfQubits = 15
ErdosRenyi = 1/3

NumberOfComponents = 0
while NumberOfComponents != 1:
    Q = np.zeros((NumberOfQubits, NumberOfQubits), dtype=int)
    for i in range(NumberOfQubits):
        for j in range(i + 1, NumberOfQubits):
            if np.random.sample() <= ErdosRenyi:
                Q[i][j] = Q[j][i] = - 1
        Q[i][i] = - np.sum(Q[i])
    NumberOfComponents = NumberOfQubits - np.linalg.matrix_rank(Q)

### this is for the normal version

In [11]:
'''
# this generates a grid
BetaGrid = 5
GammaGrid = 5
p = 5

grid = []
for beta in range(1, BetaGrid):
    for gamma in range(1, GammaGrid):
        grid.append([beta, gamma])

GridPaths = []
for pair in grid:
    GridPaths.append([pair])


delta = 1

for i in range(1, p):
    NewGridPaths = []
    for angles in GridPaths:
        for pair in grid:
            d = [(pair[0] - angles[-1][0])%BetaGrid, (pair[1] - angles[-1][1])%GammaGrid]
            if abs(d[0]) + abs(d[1]) <= delta:
                NewGridPaths.append(angles + [pair])
    GridPaths = NewGridPaths 

'''

'\n# this generates a grid\nBetaGrid = 5\nGammaGrid = 5\np = 5\n\ngrid = []\nfor beta in range(1, BetaGrid):\n    for gamma in range(1, GammaGrid):\n        grid.append([beta, gamma])\n\nGridPaths = []\nfor pair in grid:\n    GridPaths.append([pair])\n\n\ndelta = 1\n\nfor i in range(1, p):\n    NewGridPaths = []\n    for angles in GridPaths:\n        for pair in grid:\n            d = [(pair[0] - angles[-1][0])%BetaGrid, (pair[1] - angles[-1][1])%GammaGrid]\n            if abs(d[0]) + abs(d[1]) <= delta:\n                NewGridPaths.append(angles + [pair])\n    GridPaths = NewGridPaths \n\n'

### this is for the Grover angle version

In [12]:
p = 20

GroverDelta = 0.01

gamma = np.sqrt(1 - 1 / pow(np.cosh(np.arccosh(1 / GroverDelta) / (2 * p + 1)), 2))

GroverAngles = [[2 * np.arctan(1 / (gamma * np.tan(2 * np.pi * j / (2 * p + 1)))), - 2 * np.arctan(1 / (gamma * np.tan(2 * np.pi * (p - j + 1) / (2 * p + 1))))] for j in range(1, p + 1)]

### QAOA

In [13]:
def QAOA(angles):
    
    QRegX = QuantumRegister(NumberOfQubits)
    CLRegX = ClassicalRegister(NumberOfQubits)
    
    QC = QuantumCircuit(QRegX, CLRegX)

    QC.h(QRegX)
    
    for beta, gamma in angles:
        # Cost bang
        for i, q in enumerate(QRegX):
            if Q[i][i] != 0:
                QC.p(Q[i][i] * 2 * gamma, q)
            for j, r in enumerate(QRegX[:i]):
                if Q[i][j] != 0:
                    QC.cp(Q[i][j] * 4 * gamma, q, r)

        # Mixer bang
        QC.rx(beta, QRegX)

    QC.measure(QRegX, CLRegX)
    
    # error probabilities
    p_reset = 0.03
    p_meas = 0.1
    p_gate1 = 0.05

    # QuantumError objects (???)
    error_reset = pauli_error([('X', p_reset), ('I', 1 - p_reset)])
    error_meas = pauli_error([('X',p_meas), ('I', 1 - p_meas)])
    error_gate1 = pauli_error([('X',p_gate1), ('I', 1 - p_gate1)])
    error_gate2 = error_gate1.tensor(error_gate1)
    
    # errors to noise model
    noise_bit_flip = NoiseModel()
    noise_bit_flip.add_all_qubit_quantum_error(error_reset, "reset")
    noise_bit_flip.add_all_qubit_quantum_error(error_meas, "measure")
    noise_bit_flip.add_all_qubit_quantum_error(error_gate1, ["u1", "u2", "u3"])
    noise_bit_flip.add_all_qubit_quantum_error(error_gate2, ["cx"])

    # simulation
    noisy_simulator = AerSimulator(noise_model=noise_bit_flip)
    transpiled_QC = transpile(QC, noisy_simulator)    
    counts = noisy_simulator.run(transpiled_QC, shots=NumberOfShots).result().get_counts(transpiled_QC)
    
    M = 0
    bestcut = 0
    bestcutconf = np.zeros(NumberOfQubits, dtype=int)
    for s in counts:
        conf = np.array([int(x) for x in reversed(s)])
        m = np.matmul(conf, np.matmul(Q, np.transpose(conf)))
        M += m * counts[s]
        if m > bestcut:
            bestcut = m
            bestcutconf = conf

    return [bestcut, M / NumberOfShots, angles, bestcutconf, counts]

### testing

In [14]:
results = [QAOA(GroverAngles)]

#results = list(map(QAOA, GridPaths))

#from multiprocessing import Pool

#with Pool() as pool:
#    results = list(pool.imap_unordered(QAOA, GridPaths))

results.sort(key = lambda x: x[1])

### classical maximal cut computation

In [15]:
MaxCut = 0
configurations = []
for x in range(1 << (NumberOfQubits - 1)):
    conf = np.array([(x >> i)&1 for i in range(NumberOfQubits)])
    cut = np.matmul(conf, np.matmul(Q, np.transpose(conf)))
    if cut > MaxCut:
        MaxCut = cut
        configurations = [conf]
    elif cut == MaxCut:
        configurations.append(conf)

### printing the results

In [18]:
s = f"\n Number of verticies\t= {NumberOfQubits}\n p\t\t\t= {p}\n ErdosRenyi\t\t= {ErdosRenyi}\n\n Number of shots\t= {NumberOfShots}\n number of edges\t= {np.trace(Q) // 2}\n MaxCut\t\t\t= {MaxCut}\n configurations\t\t= {configurations}\n\n"
s += f" BestCut\t\t= {results[-1][0]}\n M\t\t\t= {results[-1][1]}\n configuration\t\t= {results[-1][3]}\n\n BestCut / MaxCut\t= {results[-1][0] / MaxCut}\n M / MaxCut\t\t= {results[-1][1] / MaxCut}\n\n\n"

#for result in results[-10:]:
#    s += f" BestCut\t= {result[0]}\n M\t\t= {result[1]}\n configuration\t= {result[3]}\n"
for row in Q:
    s += f"\t{row.view()}\n"

print(s)

for s in results[-1][-1]:
    conf = np.array([int(x) for x in reversed(s)])
    print(f" {conf} = {results[-1][-1][s]}\t cut = {np.matmul(conf, np.matmul(Q, np.transpose(conf)))}")

#with open('results.txt', 'w') as f:
#    f.write(s)


 Number of verticies	= 15
 p			= 20
 ErdosRenyi		= 0.3333333333333333

 Number of shots	= 10
 number of edges	= 43
 MaxCut			= 31
 configurations		= [array([1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0]), array([1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0]), array([0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0]), array([1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0]), array([0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0]), array([1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0]), array([0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0]), array([0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0])]

 BestCut		= 26
 M			= 21.0
 configuration		= [0 0 1 1 0 0 0 0 0 0 1 0 1 1 1]

 BestCut / MaxCut	= 0.8387096774193549
 M / MaxCut		= 0.6774193548387096


	[ 6 -1  0  0  0  0 -1  0  0 -1 -1  0 -1 -1  0]
	[-1  6 -1  0  0  0  0  0  0 -1 -1  0 -1  0 -1]
	[ 0 -1  6  0  0 -1  0 -1 -1  0  0 -1  0 -1  0]
	[ 0  0  0  4  0  0 -1  0 -1 -1  0  0 -1  0  0]
	[ 0  0  0  0  3  0  0 -1  0  0  0 -1  0  0 -1]
	[ 0  0 -1  0  0  4  0 